# Agrupamento de features categóricas - Implantação

Este componente realiza o agrupamento de features categóricas com alta cardinalidade para serem implementadas por modelos de classificação. Neste componente são implementadas três estratégias de agrupamento (method):

* percent: o agrupamento é feito considerando um ponto de corte definido. Categorias abaixo desse limiar são agrupadas em uma categoria denominada "other";

* top_n: apenas as n categorias mais frequentes são mantidas. As demais são agrupadas em uma categoria denominada "other";

* kmeans: Para cada classe, são calculadas sua frequência relativa e porcentagem de casos positivos da variável resposta. Então, o algoritmo kmeans utiliza estas informações para o agrupamento. 

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class ApplyGroupCatFeatures:
    def __init__(self, columns_changed=None, columns=None, dict_class={}):
        # Input variables
        self.columns_changed = columns_changed
        self.dict_class = dict_class
        self.columns = columns

    def apply_group(self, X):
        cat_columns = self.columns_changed
        X_cat = X[cat_columns]

        for col in cat_columns:
            X_cat.loc[:, col] = self.dict_class[col + "....grouped"][
                self.dict_class[col].index(X.loc[:, col].to_numpy())
            ]

        columns_tocopy = np.setdiff1d(self.columns, cat_columns)

        X_res = pd.concat([X_cat, X[columns_tocopy]], axis=1)

        return X_res


class Model(object):
    def __init__(self):
        self.loaded = False
    
    def load(self):
        # Following links explain why to use load() method insted of __init__()

        # Issue associated with sheldon on __init__
        # https://github.com/SeldonIO/seldon-core/issues/2616

        # Solution for the case
        # https://docs.seldon.io/projects/seldon-core/en/latest/python/python_component.html#gunicorn-and-load

        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/grouping-categorical-features.joblib")
        self.columns = artifacts["columns"]
        self.columns_changed = artifacts["columns_changed"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]
        self.dict_class = artifacts["dict_class"]

    def class_names(self):
        return self.features_after_pipeline.tolist()

    def predict(self, X, feature_names, meta=None):
        # First time load model
        if not self.loaded:
            self.load()

        # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
        X = pd.DataFrame(X, columns=feature_names)

        g = ApplyGroupCatFeatures(
            columns_changed=self.columns_changed,
            columns=self.columns,
            dict_class=self.dict_class,
        )
        # Realiza a transformação
        X_res = g.apply_group(X)

        return X_res.to_numpy()